# 进阶：音色语调笑声停顿

sb 2024-6-11

## 导入工具包

In [1]:
import torchaudio
import torch
from ChatTTS import ChatTTS
import soundfile
from IPython.display import Audio

## 载入预训练模型

In [2]:
chat = ChatTTS.Chat()

In [3]:
# 加载默认下载的模型
chat.load(compile=False) # 设置为Flase获得更快速度，设置为True获得更佳效果

True

## 音色

In [4]:
# 随机产生一个音色
speaker = chat.sample_random_speaker()

In [12]:
print(speaker) # save it for later timbre recovery


tensor([ 2.1493e+00,  3.3430e+00, -1.0523e+01,  5.1482e+00,  1.5392e+00,
        -4.8788e-01,  9.1713e+00,  2.6325e+00,  1.4500e+00,  2.8850e+01,
        -4.6665e-01, -3.2022e+00,  4.0992e+00,  6.9091e-02, -5.7362e+00,
         1.1225e+00,  2.3065e+00, -4.8568e+00,  2.8630e+00, -1.1725e+00,
        -5.2941e+00, -3.9418e+00,  1.2699e+00,  1.0012e+01, -4.8766e-01,
         3.2315e+00, -1.3597e+01, -1.6393e+01, -1.7456e+00, -1.2976e-02,
        -4.5303e+00,  2.9802e+00, -4.6062e-01, -6.0803e-01, -5.5618e-02,
        -2.3999e+01, -2.5776e-01,  4.1042e+00, -7.8814e+00,  4.9592e-01,
         1.6115e+00,  3.8837e+00,  8.3468e+00, -3.6279e+00, -6.3356e+00,
        -2.2932e+00,  5.1665e-01, -1.0883e+00, -4.1126e+00,  7.7323e-01,
        -2.9308e+00, -1.3367e-01, -2.0531e+00, -6.9075e-01,  6.5799e-01,
         1.4235e+00,  3.8586e+00,  4.2199e+00, -1.9207e-01,  2.6979e+00,
        -2.9687e+00, -4.1803e+00,  3.3426e+00,  3.8097e+00, -1.0637e+00,
        -1.9094e+00,  1.0241e+00,  1.8635e+01,  4.5

In [6]:
# 保存当前音色
torch.save(speaker, 'speaker/speaker_6.pth')

In [7]:
# 载入保存好的音色
speaker = torch.load('speaker/speaker_5_girl.pth')

## 句子全局设置：讲话人音色和速度

In [13]:
params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_emb = speaker, # add sampled speaker 
    temperature = .3,   # using custom temperature
    top_P = 0.7,        # top P decode
    top_K = 20,         # top K decode
)


## 句子全局设置：口语连接、笑声、停顿程度

In [16]:
# oral：连接词，AI可能会自己加字，取值范围 0-9，比如：卡壳、嘴瓢、嗯、啊、就是之类的词
# laugh：笑，取值范围 0-9
# break：停顿，取值范围 0-9

# use oral_(0-9), laugh_(0-2), break_(0-7)
params_refine_text = ChatTTS.Chat.RefineTextParams(
    prompt='[oral_6][laugh_3][break_3]'
) 

## 输入需语音合成的文字

In [17]:
texts = [
    '[laugh] 在别人眼里 [uv_break] 你是黑寡妇，[uv_break]在我这儿 [uv_break] 你就是 [uv_break] 吉祥物 [laugh][laugh]',
    '学人工智能，应该从 [uv_break] Python编程开始。经过很多年 [laugh] 的努力，才能成为一名合格的 [laugh] AI算法工程师。只想赚块钱 [laugh] 和割韭菜的人，[laugh] 是走不远的',
    'You should [uv_break] learn Python coding first [laugh] to become a super AI master. [laugh]'
]

## 语音合成模型推理

In [18]:
wavs = chat.infer(texts, params_refine_text=params_refine_text, params_infer_code=params_infer_code)

text:   0%|          | 1/384(max) [00:00,  9.64it/s]We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
text:  19%|█▉        | 74/384(max) [00:00, 128.85it/s]
code:  31%|███       | 626/2048(max) [00:03, 161.55it/s]


## 试听

In [19]:
Audio(wavs[0], rate=24000)

In [20]:
Audio(wavs[1], rate=24000)

In [21]:
Audio(wavs[2], rate=24000)

## 导出保存为音频文件

In [22]:
torchaudio.save("output/output_e1.wav", torch.from_numpy(wavs[1]).unsqueeze(0), 24000)